In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%cd "../.."

/home/hazzu/Code/opencv_streamlit


In [3]:
import os
import cv2
import numpy as np
from typing import Tuple, List
from services.semantic_keypoint_detection.services import DATATYPES


def read_image(type: int, name: str):
    """
    Reads an image and its ground truth keypoints from the specified dataset.

    :param type: The type of the dataset.
    :param name: The name of the image.

    :return (image, ground_truth): A tuple containing the image and the ground truth keypoints.
    """
    image = cv2.imread(os.path.join(DATATYPES[type], "images", f"{name}.png"))
    ground_truth = np.load(os.path.join(DATATYPES[type], "points", f"{name}.npy"))
    ground_truth = [cv2.KeyPoint(y, x, 1, 0, 0, 0) for x, y in ground_truth]
    return (image, ground_truth)


def rotate_image(image: cv2.typing.MatLike, angle: int) -> cv2.typing.MatLike:
    """
    Rotates an image by a given angle.

    :param image: The image to rotate.
    :param angle: The angle to rotate the image by.

    :return rotated_image: The rotated image.
    """
    h, w = image.shape[:2]
    matrix_rotation = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, matrix_rotation, (w, h))
    return rotated_image


def rotate_keypoints(
    size: Tuple[int, int], keypoints: List[cv2.KeyPoint], angle: int
) -> List[cv2.KeyPoint]:
    """
    Rotates the keypoints of an image.

    :param size: The size of the image (witdh, height).
    :param keypoints: The keypoints to rotate.
    :param angle: The angle to rotate the keypoints by.

    :return (result, idx): A tuple containing the rotated keypoints and their indices in the original list.
    """
    matrix_rotation = cv2.getRotationMatrix2D((size[0] / 2, size[1] / 2), angle, 1)
    kps = np.array([[kp.pt[0], kp.pt[1]] for kp in keypoints])
    kps = np.concatenate([kps, np.ones((len(kps), 1))], axis=1)
    rotated_kps = np.array(np.dot(matrix_rotation, kps.T)).T

    result, idx = [], []
    for i in range(len(rotated_kps)):
        kp = rotated_kps[i]
        if 0 <= kp[0] < size[0] and 0 <= kp[1] < size[1]:
            result.append(cv2.KeyPoint(kp[0], kp[1], 1, 0, 0, 0))
            idx.append(i)

    return (result, idx)

In [4]:
sift = cv2.SIFT_create()
bf_sift = cv2.BFMatcher()

for type in [0, 1, 3, 4, 5, 6, 7]:
    image, ground_truth = read_image(type, "1")
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    for angle in [10]:
        rotated_image = rotate_image(image, angle)
        rotated_gray_scale = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2GRAY)

        h, w = rotated_gray_scale.shape
        rotated_keypoints, idx = rotate_keypoints((w, h), ground_truth, angle)

        original_keypoints = [ground_truth[i] for i in idx]
        original_descriptors = sift.compute(gray_scale, original_keypoints)[1]
        rotated_descriptors = sift.compute(rotated_gray_scale, rotated_keypoints)[1]

        matches = bf_sift.knnMatch(original_descriptors, rotated_descriptors, k=2)
        if len(matches) == 0 or len(matches[0]) < 2:
            continue

        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance and m.trainIdx == m.queryIdx:
                good_matches.append(m)

        image_matches = cv2.drawMatches(
            image, ground_truth, rotated_image, rotated_keypoints, good_matches, None
        )
        cv2.imwrite(
            f"./services/keypoint_matching/results/sift_{type}_{angle}.png",
            image_matches,
        )
        np.save(
            f"./services/keypoint_matching/results/sift_{type}_{angle}.npy",
            np.array([len(good_matches), len(original_keypoints)]),
        )

In [5]:
orb = cv2.ORB_create()
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

for type in [0, 1, 3, 4, 5, 6, 7]:
    image, ground_truth = read_image(type, "1")
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    for angle in [10]:
        rotated_image = rotate_image(image, angle)
        rotated_gray_scale = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2GRAY)

        h, w = rotated_gray_scale.shape
        rotated_keypoints, idx = rotate_keypoints((w, h), ground_truth, angle)

        original_keypoints = [ground_truth[i] for i in idx]
        original_descriptors = orb.compute(gray_scale, original_keypoints)[1]
        rotated_descriptors = orb.compute(rotated_gray_scale, rotated_keypoints)[1]

        matches = bf_orb.match(original_descriptors, rotated_descriptors)
        matches = [m for m in matches if m.queryIdx == m.trainIdx]
        image_matches = cv2.drawMatches(
            image, ground_truth, rotated_image, rotated_keypoints, matches, None
        )

        cv2.imwrite(
            f"./services/keypoint_matching/results/orb_{type}_{angle}.png",
            image_matches,
        )
        np.save(
            f"./services/keypoint_matching/results/orb_{type}_{angle}.npy",
            np.array([len(matches), len(original_keypoints)]),
        )

In [6]:
from services.keypoint_matching.superpoint import SuperPointFrontend


fe = SuperPointFrontend(
    weights_path="services/keypoint_matching/superpoint_v1.pth",
    nms_dist=4,
    conf_thresh=0.015,
    nn_thresh=0.7,
    cuda=True,
)
bf_fe = cv2.BFMatcher()

for type in [0, 1, 3, 4, 5, 6, 7]:
    image, ground_truth = read_image(type, "1")

    for angle in [10]:
        rotated_image = rotate_image(image, angle)
        rotated_gray_scale = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2GRAY)
        rotated_gray_scale = rotated_gray_scale.astype(np.float32) / 255.0

        h, w = rotated_gray_scale.shape
        rotated_keypoints, idx = rotate_keypoints((w, h), ground_truth, angle)

        original_keypoints = [ground_truth[i] for i in idx]
        original_descriptors = fe.compute(rotated_gray_scale, original_keypoints)[1]
        rotated_descriptors = fe.compute(rotated_gray_scale, rotated_keypoints)[1]

        matches = bf_fe.knnMatch(original_descriptors, rotated_descriptors, k=2)
        if len(matches) == 0 or len(matches[0]) < 2:
            continue

        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance and m.trainIdx == m.queryIdx:
                good_matches.append(m)

        image_matches = cv2.drawMatches(
            image, ground_truth, rotated_image, rotated_keypoints, good_matches, None
        )
        cv2.imwrite(
            f"./services/keypoint_matching/results/superpoint_{type}_{angle}.png",
            image_matches,
        )
        np.save(
            f"./services/keypoint_matching/results/superpoint_{type}_{angle}.npy",
            np.array([len(good_matches), len(original_keypoints)]),
        )

/home/hazzu/Code/opencv_streamlit/services/keypoint_matching/superpoint.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(weights_path

Run on GPU


/home/hazzu/anaconda3/envs/opencv_streamlit/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
